In [1]:
import numpy as np
import pandas as pd

dc_listings = pd.read_csv("dc_airbnb.csv")
stripped_commas = dc_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')

shuffeled_index = np.random.permutation(len(dc_listings))
dc_listings = dc_listings.iloc[shuffeled_index]
split_one = dc_listings[0:1862]
split_two = dc_listings[1862:]


In [3]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

train_one = split_one
test_one = split_two
train_two = split_two
test_two = split_one

knn = KNeighborsRegressor()
knn.fit(train_one[['accommodates']], train_one['price'])
predictions = knn.predict(test_one[['accommodates']])
iteration_one_rmse = (mean_squared_error(test_one['price'], predictions)) ** (1/2)

knn = KNeighborsRegressor()
knn.fit(train_two[['accommodates']], train_two['price'])
predictions = knn.predict(test_two[['accommodates']])
iteration_two_rmse = (mean_squared_error(test_two['price'], predictions)) ** (1/2)

avg_rmse = np.mean([iteration_one_rmse, iteration_two_rmse])

In [4]:
avg_rmse

135.89441363137385

In [5]:

dc_listings.set_value(dc_listings.index[0:744], "fold", 1)
dc_listings.set_value(dc_listings.index[744:1488], "fold", 2)
dc_listings.set_value(dc_listings.index[1488:2232], "fold", 3)
dc_listings.set_value(dc_listings.index[2232:2976], "fold", 4)
dc_listings.set_value(dc_listings.index[2976:3722], "fold", 5)


dc_listings['fold'].value_counts()

5.0    746
4.0    744
3.0    744
2.0    744
1.0    744
Name: fold, dtype: int64

In [7]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

train = dc_listings[dc_listings['fold'] != 1]
test =  dc_listings[dc_listings['fold'] == 1]
knn = KNeighborsRegressor()
knn.fit(train[['accommodates']], train['price'])
predictions = knn.predict(test[['accommodates']])
iteration_one_rmse = (mean_squared_error(test['price'], predictions))**(1/2)
iteration_one_rmse

160.71821497159527

In [8]:
# Use np.mean to calculate the mean.
import numpy as np
fold_ids = [1,2,3,4,5]

def train_and_validate(df, folds):
    rmses = []
    for f in folds:
        train = df[df['fold'] != f]
        test =  df[df['fold'] == f]
        knn = KNeighborsRegressor()
        knn.fit(train[['accommodates']], train['price'])
        predictions = knn.predict(test[['accommodates']])
        rmses.append((mean_squared_error(test['price'], predictions))**(1/2))
    return rmses

rmses = train_and_validate(dc_listings, fold_ids)
print(rmses)
avg_rmse = np.average(rmses)
print(avg_rmse)

[160.71821497159527, 126.58777905647769, 104.42053854950061, 103.92419745759069, 134.83032709504803]
126.096211426


In [9]:
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score

kf = KFold(len(dc_listings), 5, shuffle=True, random_state=1)
knn = KNeighborsRegressor()
mses = cross_val_score(knn, dc_listings[['accommodates']], dc_listings['price'], cv=kf, scoring='mean_squared_error')
print(mses)
rmses = [abs(x) ** (1/2) for x in mses]
print(rmses)
avg_rmse = np.average(rmses)
avg_rmse

[-14586.6338255  -21762.84375839 -16737.46577181  -9012.98010753
 -23842.15591398]
[120.77513744766908, 147.52235003005228, 129.373358044893, 94.936716330021028, 154.40905386012341]


/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda/lib/python3.6/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda/lib/python3.6/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda/lib/python3.6/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring metho

129.40332314255176

The scikit-learn version is 0.18.1.
